In [13]:
!git clone https://github.com/KISS/tweets-to-emoji.git # Dataset, may require execution when re-login to colab
!pip install gingerit
!pip install contractions
!pip install emoji
!pip install spellchecker

Cloning into 'tweets-to-emoji'...
remote: Enumerating objects: 732, done.
remote: Counting objects: 100% (732/732), done.
remote: Compressing objects: 100% (294/294), done.
remote: Total 732 (delta 436), reused 729 (delta 436), pack-reused 0
Receiving objects: 100% (732/732), 6.60 MiB | 13.62 MiB/s, done.
Resolving deltas: 100% (436/436), done.
     |████████████████████████████████| 245kB 5.5MB/s 
     |████████████████████████████████| 317kB 13.4MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81695 sha256=44e1547984f7112ce35f06f096c41b4ac41f57717bfa23d16f185945eecfade9
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [14]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import nltk as nl
nl.download('punkt')
import contractions
from sklearn.model_selection import train_test_split 
import emoji
from gingerit.gingerit import GingerIt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Load the data set

In [17]:
data = pd.read_csv("/content/tweets-to-emoji/csv/all_tweets.csv", dtype=np.str, delimiter=",") # Whole dataset
sentences=data['text']

Clean every sentence by deleting URL, correcting misspelling, replacing contents like I'm wtih I am, deleting non-alphabet characters. 

In [ ]:
cleanedSentences = []
parser = GingerIt() #uesd for corrcect misspelling
regex = re.compile('[^a-zA-z0-9]')#uesd for deleting non-alphabet characters
for sentence in sentences:
  URLless_string = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}     /)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', sentence)
  nomisspelling = parser.parse(URLless_string)["text"]
  contfix = contractions.fix(nomisspelling)
  AlphabetOnly = regex.sub(' ', contfix) 
  #cleanedSentences.append(AlphabetOnly)
  print(AlphabetOnly)
cleanedSentences

Streaming output truncated to the last 5000 lines.
ur phone does not Autocorrect while ur in all CAPS because it thinks ur angry and does not want to get involved 
 I will put that down as a  maybe     hilarious   Well done Sir   with any luck they will not be darkening your doorstep  
Seeing Jim Bollman talking to the starting freshmen offensive linemen just made me irrationally angry 
The Angry Birds Move 2  4K UHD Blu ray Review  
everytime i see those hags on my tl i get angry like bro disband already PLEASE
I am so angry 
if this was possible it would make me so angry and so happy at the same time 
Hi I will be angry drinking until we either A  turn our ass around and start playing better Or  B  die  Thank you for your time
like rly angry
It would be extremely good if everyone in Brian Dennehy was leaving angry in the 3rd quarter   Doubt it happens  But would be neat 
Messi wants to kill these people  he is really angry over the last match I guess
I miss the moment when you smile 

In [ ]:
tokens = []
for sentence in cleanedSentences: 
  tokens.append(nl.word_tokenize(sentence))
tokens=np.array(tokens)
tokens[data['filtered_for']=='afraid']

array([list(['A', 'reminder', 'of', 'how', 'unacceptable', 'it', 'was', 'when', 'Barr', 'admitted', 'he', 'did', 'not', 'review', 'the', 'underlying', 'evidence', 'in', 'Mueller', 's', 'inve']),
       list(['The', 'libs', 'are', 'terrified', 'about', 'the', 'content', 'of', 'my', 'shows', 'Spread', 'this', 'show', 'far', 'and', 'wide', 'to', 'see', 'what', 'the', 'YouTube', 'snowfla']),
       list(['One', 'billionaire', 'afraid', 'of', 'spreading', 'populism', 'and', 'calls', 'to', 'tax', 'him', 'heavily', 'and', 'break', 'up', 'his', 'company', 'called', 'another']),
       ...,
       list(['Be', 'not', 'afraid', 'of', 'growing', 'slowly', 'be', 'afraid', 'only', 'of', 'standing', 'still', 'Chinese', 'Proverb']),
       list(['I', 'am', 'afraid', 'of', 'taking', 'a', 'laxative', 'I', 'am', 'just', 'going', 'to', 'get', 'my', 'ass', 'back', 'into', 'the', 'gym', 'and', 'kitchen']),
       list(['I', 'am', 'not', 'afraid', 'of', 'The', 'Me', 'Too', 'Movement', 'because', 'my', 'shit'

In [ ]:
label = np.array(data['filtered_for'])

print(emoji.emojize(" :red_heart:	", use_aliases=True))
emoji.emojize(":winking_face_with_tongue:") 


 ❤	


'😜'

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(label)
print(integer_encoded)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded[13950,:])

inverted = label_encoder.inverse_transform([np.argmax(onehot_encoded[0, :])])
print(inverted)

[ 0  0  0 ... 21 21 21]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
['afraid']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tokens,onehot_encoded, test_size=0.2, random_state=42)
print(X_train.shape,y_train.shape)



(28844,) (28844, 22)


In [ ]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)

        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map


word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('/content/tweets-to-emoji/glove.6B.50d.txt')

In [ ]:
word = "cucumber"
word = "hawaii"
idx = 289846
print("the index of", word, "in the vocabulary is", word_to_index[word])
#print("the", str(idx) + "th word in the vocabulary is", index_to_word[idx])
word_to_vec_map[word]

the index of hawaii in the vocabulary is 174730


array([-1.4040e-01,  1.5008e-01, -4.9972e-01,  1.0246e+00, -5.3435e-01,
       -5.5254e-01, -1.0698e+00,  2.4338e-01,  4.9893e-01, -6.2654e-01,
        3.1192e-02, -7.8421e-02,  1.7942e+00,  2.4433e-01,  6.6174e-04,
        4.1294e-01, -8.2945e-01,  4.7115e-01, -1.2470e+00,  6.0569e-01,
       -3.7575e-02,  1.6532e-01, -2.5539e-02, -2.2036e-01,  5.2229e-01,
       -1.4994e+00,  7.6258e-02, -4.2083e-01, -1.4400e-01, -7.9049e-01,
        1.0349e+00,  2.1128e-01, -7.1156e-01, -4.5174e-01,  4.3308e-01,
       -5.2842e-01, -3.6669e-01, -2.4261e-01, -2.7308e-01, -3.6893e-01,
       -1.3955e+00,  1.9290e-01,  6.8895e-01,  1.1954e-01, -6.5048e-01,
        4.1567e-01,  1.5023e-01, -1.1001e+00, -5.7182e-01,  4.8958e-01])

In [ ]:
def sentence_to_avg(sentence, word_to_vec_map):
    """
    Converts a sentence (string) into a list of words (strings). Extracts the GloVe representation of each word
    and averages its value into a single vector encoding the meaning of the sentence.
    
    Arguments:
    sentence -- string, one training example from X
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    
    Returns:
    avg -- average vector encoding information about the sentence, numpy-array of shape (50,)
    """
    
    ### START CODE HERE ###
    # Step 1: Split sentence into list of lower case words (≈ 1 line)
    words = sentence.lower().split()

    # Initialize the average word vector, should have the same shape as your word vectors.
    avg = np.zeros(50,)
    
    # Step 2: average the word vectors. You can loop over the words in the list "words".
    total = 0
    for w in words:
        #print(w)
        if w in word_to_vec_map.keys():
          total += word_to_vec_map[w]
    avg = total/len(words)
    
    ### END CODE HERE ###
    
    return avg

avg = sentence_to_avg("Morrocan couscous is my favorite dish", word_to_vec_map)
print("avg = \n", avg)

avg = 
 [-0.008005    0.56370833 -0.50427333  0.258865    0.55131103  0.03104983
 -0.21013718  0.16893933 -0.09590267  0.141784   -0.15708967  0.18525867
  0.6495785   0.38371117  0.21102167  0.11301667  0.02613967  0.26037767
  0.05820667 -0.01578167 -0.12078833 -0.02471267  0.4128455   0.5152061
  0.38756167 -0.898661   -0.535145    0.33501167  0.68806933 -0.2156265
  1.797155    0.10476933 -0.36775333  0.750785    0.10282583  0.348925
 -0.27262833  0.66768    -0.10706167 -0.283635    0.59580117  0.28747333
 -0.3366635   0.23393817  0.34349183  0.178405    0.1166155  -0.076433
  0.1445417   0.09808667]


In [ ]:
input_vec=np.zeros((X_train.shape[0],50))
for idx in range(X_train.shape[0]):
  input_vec[idx,:] = sentence_to_avg(X_train[idx][0], word_to_vec_map)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
# fix random seed for reproducibility
np.random.seed(7)

In [6]:
model = Sequential()
model.add(Dense(20, input_dim=50, activation='relu'))
model.add(Dense(1, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(input_vec, y_train, epochs=150, batch_size=10)

NameError: ignored

In [11]:
from gingerit.gingerit import GingerIt

text = 'The smelt of fliwers bring back memories.'

parser = GingerIt()
parser.parse(text)

{'corrections': [{'correct': 'brings',
   'definition': None,
   'start': 21,
   'text': 'bring'},
  {'correct': 'flowers',
   'definition': 'a plant cultivated for its blooms or blossoms',
   'start': 13,
   'text': 'fliwers'},
  {'correct': 'smell', 'definition': None, 'start': 4, 'text': 'smelt'}],
 'result': 'The smell of flowers brings back memories.',
 'text': 'The smelt of fliwers bring back memories.'}

In [ ]:
!git clone https://github.com/neonrights/emoji_predictor.git